# Fantasy Injury Risk Data Imports and First Look

## Imports

In [8]:
# Libraries and output settings
import nfl_data_py as nfl
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Import weekly injuries
injuries = nfl.import_injuries(range(2009,2025))

# Import seasonal rosters
rosters_seas = nfl.import_seasonal_rosters(range(2009, 2025))

# Import weekly rosters
rosters_weekly = nfl.import_weekly_rosters(range(2009,2025))

# Import seasonal stats
stats_seas = nfl.import_seasonal_data(range(2009,2025), 'ALL')

## Explore Data

**Start with which positions are in dataset**

In [9]:
# Position value counts
rosters_seas['position'].value_counts()

DB     5472
WR     5258
OL     4726
DL     4109
LB     3827
RB     3263
TE     2806
QB     1838
CB     1438
DE     1133
OLB    1125
T      1002
G       895
K       769
DT      750
P       676
LS      661
SS      581
FS      550
ILB     516
C       479
NT      317
MLB     291
FB      275
S        32
PR        2
KR        2
Name: position, dtype: int64

**Datasets contain information for player positions that are not fantasy football positions. Fantasy positions are QB, RB, WR, TE and K. Need to condense down to relevant positions**

Addtionally, fullback is a position that is rarely used anymore, but is equivalent to runningback for fantasy purposes. Will retitle in datasets.

In [10]:
# Make list of dfs to use in loop to replace FB with RB
dfs = [rosters_seas, rosters_weekly, injuries]

# Loop over dfs and replace
for df in dfs:
  df['position'] = df['position'].replace({'FB' : 'RB'})

In [11]:
# Filter dfs to only show fantasy relevant positions
rosters_weekly_ff = rosters_weekly[rosters_weekly['position'].isin(['RB', 'QB', 'TE', 'WR', 'K'])]
rosters_seas_ff = rosters_seas[rosters_seas['position'].isin(['RB', 'QB', 'TE', 'WR', 'K'])]
injuries_ff = injuries[injuries['position'].isin(['RB', 'QB', 'TE', 'WR', 'K'])]


### Info checks

Looking to explore what unique IDs are available for eventual merge of datasets

#### Injuries

Available unique ID - Number of nulls:
1. gsis_id - 0

In [12]:
# Injuries info
injuries_ff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27187 entries, 0 to 6213
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   season                     27187 non-null  float64       
 1   game_type                  27187 non-null  object        
 2   team                       27187 non-null  object        
 3   week                       27187 non-null  float64       
 4   gsis_id                    27187 non-null  object        
 5   position                   27187 non-null  object        
 6   full_name                  27187 non-null  object        
 7   first_name                 27187 non-null  object        
 8   last_name                  27187 non-null  object        
 9   report_primary_injury      18671 non-null  object        
 10  report_secondary_injury    1103 non-null   object        
 11  report_status              18674 non-null  object        
 12  pract

#### Seasonal Roster

Available unique ID - number of nulls:
1. player_id - 0
2. espn_id - 3358
3. sportradar_id - 2999
4. yahoo_id - 3976
5. rotowire_id - 2961
6. pff_id - 4096
7. pfr_id - 5830
8. fantasy_data_id - 4751
9. sleeper_id - 3483
10. esb_id - 14
11. gsis_it_id - 2272
12. smart_id - 44

In [13]:
# Seasonal info
rosters_seas_ff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14209 entries, 1 to 42818
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   season                   14209 non-null  int32         
 1   team                     14209 non-null  object        
 2   position                 14209 non-null  object        
 3   depth_chart_position     9315 non-null   object        
 4   jersey_number            14127 non-null  object        
 5   status                   14206 non-null  object        
 6   player_name              14209 non-null  object        
 7   first_name               14209 non-null  object        
 8   last_name                14209 non-null  object        
 9   birth_date               13317 non-null  datetime64[ns]
 10  height                   14207 non-null  float64       
 11  weight                   14207 non-null  float64       
 12  college                  11670 n

**Check how many ID types are missing to see which ones are missing teh least**

In [14]:
# Create list of column names that contain ID for loop
ids = rosters_seas_ff.columns[rosters_seas_ff.columns.str.contains('id', case=False)].tolist()

# Loop over ID columns and return NaNs for each rtpe
for i, id in enumerate(ids, 1):
  print(f'{i}. {id} - {rosters_seas_ff[id].isna().sum()}')

1. player_id - 0
2. espn_id - 3535
3. sportradar_id - 3180
4. yahoo_id - 4181
5. rotowire_id - 3138
6. pff_id - 4273
7. pfr_id - 6056
8. fantasy_data_id - 4944
9. sleeper_id - 3676
10. esb_id - 14
11. gsis_it_id - 2450
12. smart_id - 44


#### Weekly Roster

Available unique ID - number of nulls:
1. player_id - 86
2. espn_id - 40216
3. sportradar_id - 34927
4. yahoo_id - 49752
5. rotowire_id - 34358
6. pff_id - 51743
7. pfr_id - 75772
8. fantasy_data_id - 62375
9. sleeper_id - 42208
10. esb_id - 33
11. gsis_it_id - 30137
12. smart_id - 227

In [15]:
# Weekly info
rosters_weekly_ff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210577 entries, 19 to 641697
Data columns (total 37 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   season                   210577 non-null  int32         
 1   team                     210577 non-null  object        
 2   position                 210577 non-null  object        
 3   depth_chart_position     138519 non-null  object        
 4   jersey_number            210279 non-null  object        
 5   status                   210573 non-null  object        
 6   player_name              210577 non-null  object        
 7   first_name               210577 non-null  object        
 8   last_name                210577 non-null  object        
 9   birth_date               201534 non-null  datetime64[ns]
 10  height                   210489 non-null  float64       
 11  weight                   210575 non-null  float64       
 12  college        

In [16]:
# Create list of ID columns for loop
ids = rosters_weekly_ff.columns[rosters_weekly_ff.columns.str.contains('id', case=False)].tolist()

# loop over ID columns and return NaNs for each type
for i, id in enumerate(ids, 1):
  print(f'{i}. {id} - {rosters_weekly_ff[id].isna().sum()}')

1. player_id - 86
2. espn_id - 42820
3. sportradar_id - 37579
4. yahoo_id - 52743
5. rotowire_id - 36962
6. pff_id - 54347
7. pfr_id - 79116
8. fantasy_data_id - 65208
9. sleeper_id - 45041
10. esb_id - 33
11. gsis_it_id - 32698
12. smart_id - 227


#### Seasonal Stats

Available unique ID - number of nulls:
1. player_id - 0

In [17]:
# Season info
stats_seas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11744 entries, 0 to 11743
Data columns (total 58 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    11744 non-null  object 
 1   season                       11744 non-null  int64  
 2   season_type                  11744 non-null  object 
 3   completions                  11744 non-null  int32  
 4   attempts                     11744 non-null  int32  
 5   passing_yards                11744 non-null  float64
 6   passing_tds                  11744 non-null  int32  
 7   interceptions                11744 non-null  float64
 8   sacks                        11744 non-null  float64
 9   sack_yards                   11744 non-null  float64
 10  sack_fumbles                 11744 non-null  int32  
 11  sack_fumbles_lost            11744 non-null  int32  
 12  passing_air_yards            11744 non-null  float64
 13  passing_yards_af

In [18]:
# Create list of ID columns for loop
ids = stats_seas.columns[stats_seas.columns.str.contains('id', case=False)].tolist()

# loop over ID columns and return NaNs for each type
for i, id in enumerate(ids, 1):
  print(f'{i}. {id} - {stats_seas[id].isna().sum()}')

1. player_id - 0


### Data Cleaning

#### Unique ID mismatches

**Across all four datasets, there are a variety of unique IDs, not all of them overlap. Additionally, the seasonal statistics do not have any player names, only player IDs.**

My plan is to use player ID from one of the datasets along with name so I can add my own unique ID to all datasets for a merge.

**Check how many unique IDs are in each dataset to see which contain the most information**

In [19]:
# Create list for loop
dfs = [stats_seas, rosters_weekly_ff, rosters_seas_ff]

# give names to each df for loop
df_names = ["stats_seas", "rosters_weekly", "rosters_seas"]

# Loop over and return number of unique IDs in each dataset
for i, (df, name) in enumerate(zip(dfs, df_names), 1):
  print(f'{i}. {name} - {df["player_id"].nunique()} unique player IDs')


1. stats_seas - 2847 unique player IDs
2. rosters_weekly - 3877 unique player IDs
3. rosters_seas - 3873 unique player IDs


**Check to see who is missing from the rosters season that are in rosters weekly**

In [20]:
# Get unique player IDs from each dataframe
weekly_ids = rosters_weekly_ff['player_id'].unique()
season_ids = rosters_seas_ff['player_id'].unique()

# Convert to sets for set difference operation
weekly_ids_set = set(weekly_ids)
season_ids_set = set(season_ids)

# Find IDs in weekly but not in season
ids_in_weekly_not_in_season = weekly_ids_set - season_ids_set

# Convert the result to a list
ids_list = list(ids_in_weekly_not_in_season)

print(f"Number of player IDs in rosters_weekly but not in rosters_seas: {len(ids_list)}")

# Create a dataframe with these IDs and their details from rosters_weekly
missing_players_df = rosters_weekly_ff[rosters_weekly_ff['player_id'].isin(ids_list)]

# To get a unique list of these players (without duplicates)
unique_missing_players_df = missing_players_df.drop_duplicates(subset=['player_id'])

# Display the resulting dataframe
print(unique_missing_players_df[['player_id', 'player_name', 'position', 'status']])

Number of player IDs in rosters_weekly but not in rosters_seas: 5
         player_id     player_name position status
247683  00-0031876       Matt Wile        K    ACT
252272  00-0032466     George Fant       TE    ACT
360831        None  Darren Andrews       WR    RSN
391417  00-0034128    Nate Wozniak       TE    CUT
448828  00-0035833  Brandon Wright        K    CUT


In [21]:
# Loop over and return head for the missing IDs
for id in ids_list:
  display(rosters_weekly_ff[rosters_weekly_ff['player_id'] == id].head())

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age


,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
448828,2020,JAX,K,K,1.0,CUT,Brandon Wright,Brandon,Wright,1997-02-08,70.0,182.0,Georgia State,00-0035833,None,61980614-1609-4b68-b11f-05aa30fefb89,33149,14954,None,None,22292,7296,0.0,None,None,4,REG,None,Brandon,WRI116406,52735,32005752-4911-6406-49fa-f1b22d3d5c4c,2020.0,2020.0,None,NaN,23.652
448829,2020,JAX,K,K,1.0,CUT,Brandon Wright,Brandon,Wright,1997-02-08,70.0,182.0,Georgia State,00-0035833,None,61980614-1609-4b68-b11f-05aa30fefb89,33149,14954,None,None,22292,7296,0.0,None,None,7,REG,None,Brandon,WRI116406,52735,32005752-4911-6406-49fa-f1b22d3d5c4c,2020.0,2020.0,None,NaN,23.710
448830,2020,JAX,K,K,1.0,CUT,Brandon Wright,Brandon,Wright,1997-02-08,70.0,182.0,Georgia State,00-0035833,None,61980614-1609-4b68-b11f-05aa30fefb89,33149,14954,None,None,22292,7296,0.0,None,None,5,REG,None,Brandon,WRI116406,52735,32005752-4911-6406-49fa-f1b22d3d5c4c,2020.0,2020.0,None,NaN,23.671
448832,2020,JAX,K,K,1.0,ACT,Brandon Wright,Brandon,Wright,1997-02-08,70.0,182.0,Georgia State,00-0035833,None,61980614-1609-4b68-b11f-05aa30fefb89,33149,14954,None,None,22292,7296,0.0,None,None,3,REG,A01,Brandon,WRI116406,52735,32005752-4911-6406-49fa-f1b22d3d5c4c,2020.0,2020.0,None,NaN,23.625
448834,2020,JAX,K,K,1.0,CUT,Brandon Wright,Brandon,Wright,1997-02-08,70.0,182.0,Georgia State,00-0035833,None,61980614-1609-4b68-b11f-05aa30fefb89,33149,14954,None,None,22292,7296,0.0,None,None,6,REG,None,Brandon,WRI116406,52735,32005752-4911-6406-49fa-f1b22d3d5c4c,2020.0,2020.0,None,NaN,23.691


,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
247683,2016,DAL,K,K,2.0,ACT,Matt Wile,Matt,Wile,1992-06-20,74.0,215.0,Michigan,00-0031876,2576240,4278baf5-f774-4031-ab0f-12a9c7e43c45,28738,10532,None,None,17550,3008,1.0,https://static.www.nfl.com/image/private/f_aut...,None,1,REG,A01,Matt,WIL048868,42907,32005749-4c04-8868-f487-a75d2479caa3,2015.0,2015.0,None,NaN,24.227
327670,2018,MIN,K,K,6.0,ACT,Matt Wile,Matt,Wile,1992-06-20,74.0,219.0,Michigan,00-0031876,2576240,4278baf5-f774-4031-ab0f-12a9c7e43c45,28738,10532,None,None,17550,3008,3.0,https://static.www.nfl.com/image/private/f_aut...,None,4,REG,A01,Matt,WIL048868,42907,32005749-4c04-8868-f487-a75d2479caa3,2015.0,2015.0,None,NaN,26.270
327673,2018,MIN,K,K,6.0,ACT,Matt Wile,Matt,Wile,1992-06-20,74.0,219.0,Michigan,00-0031876,2576240,4278baf5-f774-4031-ab0f-12a9c7e43c45,28738,10532,None,None,17550,3008,3.0,https://static.www.nfl.com/image/private/f_aut...,None,5,REG,A01,Matt,WIL048868,42907,32005749-4c04-8868-f487-a75d2479caa3,2015.0,2015.0,None,NaN,26.297
327677,2018,MIN,K,K,6.0,ACT,Matt Wile,Matt,Wile,1992-06-20,74.0,219.0,Michigan,00-0031876,2576240,4278baf5-f774-4031-ab0f-12a9c7e43c45,28738,10532,None,None,17550,3008,3.0,https://static.www.nfl.com/image/private/f_aut...,None,1,REG,A01,Matt,WIL048868,42907,32005749-4c04-8868-f487-a75d2479caa3,2015.0,2015.0,None,NaN,26.220
327678,2018,MIN,K,K,6.0,ACT,Matt Wile,Matt,Wile,1992-06-20,74.0,219.0,Michigan,00-0031876,2576240,4278baf5-f774-4031-ab0f-12a9c7e43c45,28738,10532,None,None,17550,3008,3.0,https://static.www.nfl.com/image/private/f_aut...,None,2,REG,A01,Matt,WIL048868,42907,32005749-4c04-8868-f487-a75d2479caa3,2015.0,2015.0,None,NaN,26.240


,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
252272,2016,SEA,TE,T,74.0,ACT,George Fant,George,Fant,1992-07-19,77.0,270.0,Western Kentucky,00-0032466,None,None,None,None,None,None,None,None,0.0,https://static.www.nfl.com/image/private/f_aut...,None,1,REG,A01,George,FAN401790,43586,32004641-4e40-1790-4ceb-d01018fe0d26,2016.0,2016.0,None,NaN,24.148


,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
391417,2019,MIN,TE,TE,79.0,CUT,Nate Wozniak,Nate,Wozniak,1994-08-30,82.0,302.0,Minnesota,00-0034128,None,None,None,None,None,None,None,None,1.0,https://static.www.nfl.com/image/private/f_aut...,None,1,REG,None,Nate,WOZ425791,46390,3200574f-5a42-5791-5d77-20b7540ca069,2018.0,2018.0,None,NaN,25.024
391420,2019,ATL,TE,TE,79.0,CUT,Nate Wozniak,Nate,Wozniak,1994-08-30,82.0,280.0,Minnesota,00-0034128,None,None,None,None,None,None,None,None,1.0,https://static.www.nfl.com/image/private/f_aut...,None,4,REG,None,Nate,WOZ425791,46390,3200574f-5a42-5791-5d77-20b7540ca069,2018.0,2018.0,None,NaN,25.081
391422,2019,ATL,TE,TE,79.0,DEV,Nate Wozniak,Nate,Wozniak,1994-08-30,82.0,280.0,Minnesota,00-0034128,None,None,None,None,None,None,None,None,1.0,https://static.www.nfl.com/image/private/f_aut...,None,3,REG,None,Nate,WOZ425791,46390,3200574f-5a42-5791-5d77-20b7540ca069,2018.0,2018.0,None,NaN,25.062
391429,2019,MIN,TE,TE,79.0,CUT,Nate Wozniak,Nate,Wozniak,1994-08-30,82.0,302.0,Minnesota,00-0034128,None,None,None,None,None,None,None,None,1.0,https://static.www.nfl.com/image/private/f_aut...,None,2,REG,None,Nate,WOZ425791,46390,3200574f-5a42-5791-5d77-20b7540ca069,2018.0,2018.0,None,NaN,25.043


**Seasonal roster less unique player IDs than weekly. In checking weekly, the 5 extra IDs are players who haven't played a single down in the NFL. Will use seasonal IDs and will ultimately end us throwing out the players who have never played a snap**

##### Create my own unique ID

I want to make sure that every player has a unqique ID that is also connected to their season, since one row will represent player/season

In [22]:
# Create dataframe with only player names and their corresponding player ID for cleaning
id_df = rosters_seas_ff[['player_name', 'player_id', 'position', 'team']]
id_df.head()

,player_name,player_id,position,team
1,David Akers,00-0000108,K,PHI
7,Charlie Batch,00-0000865,QB,PIT
13,Marty Booker,00-0001477,WR,ATL
16,Kris Brown,00-0001980,K,HST
17,Isaac Bruce,00-0002099,WR,SF


**In later data review, some players have different positions attached to their names. Each player should only have one position, take a look**

In [23]:
# Look at player who has different positions
id_df[id_df['player_id'] == '00-0026019']

,player_name,player_id,position,team
1507,Danny Woodhead,00-0026019,WR,NYJ
3322,Danny Woodhead,00-0026019,WR,NE
3323,Danny Woodhead,00-0026019,WR,NYJ
5191,Danny Woodhead,00-0026019,RB,NE
7037,Danny Woodhead,00-0026019,RB,NE
8980,Danny Woodhead,00-0026019,RB,SD
10981,Danny Woodhead,00-0026019,RB,SD
13035,Danny Woodhead,00-0026019,RB,SD
15185,Danny Woodhead,00-0026019,RB,SD
18142,Danny Woodhead,00-0026019,RB,BAL


**Take the most common position for each player ID and apply to dataset**

In [24]:
# Group by player_id and find the most common position, then apply to that player ID
comm_pos = id_df.groupby('player_id')['position'].agg(lambda x: x.mode()[0])
comm_pos

player_id
00-0000108     K
00-0000865    QB
00-0001477    WR
00-0001545    QB
00-0001980     K
              ..
00-0039940    RB
00-0039941    TE
00-0039942    TE
00-0039945     K
00-0039952    TE
Name: position, Length: 3873, dtype: object

In [25]:
# Create dictionary of positions and ID and map to player ID
pos_dict = comm_pos.to_dict()
id_df['std_pos'] = id_df['player_id'].map(pos_dict)
id_df

C:\Users\nick_\AppData\Local\Temp\ipykernel_22324\648753905.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_df['std_pos'] = id_df['player_id'].map(pos_dict)


,player_name,player_id,position,team,std_pos
1,David Akers,00-0000108,K,PHI,K
7,Charlie Batch,00-0000865,QB,PIT,QB
13,Marty Booker,00-0001477,WR,ATL,WR
16,Kris Brown,00-0001980,K,HST,K
17,Isaac Bruce,00-0002099,WR,SF,WR
...,...,...,...,...,...
42806,Kairee Robinson,00-0039940,RB,SEA,RB
42807,Geor'Quarius Spivey,00-0039941,TE,KC,TE
42808,Mason Fairchild,00-0039942,TE,NO,TE
42811,Alex Hale,00-0039945,K,GB,K


In [26]:
# Check that std_pos actually shows the same for the players with different position
id_df[id_df['player_id'] == '00-0026019']

,player_name,player_id,position,team,std_pos
1507,Danny Woodhead,00-0026019,WR,NYJ,RB
3322,Danny Woodhead,00-0026019,WR,NE,RB
3323,Danny Woodhead,00-0026019,WR,NYJ,RB
5191,Danny Woodhead,00-0026019,RB,NE,RB
7037,Danny Woodhead,00-0026019,RB,NE,RB
8980,Danny Woodhead,00-0026019,RB,SD,RB
10981,Danny Woodhead,00-0026019,RB,SD,RB
13035,Danny Woodhead,00-0026019,RB,SD,RB
15185,Danny Woodhead,00-0026019,RB,SD,RB
18142,Danny Woodhead,00-0026019,RB,BAL,RB


In [27]:
# Make sure unique IDS and player IDs have same number of unique values
print(f'Player name nunique - {id_df["player_name"].nunique()}')
print(f'Player ID nunique - {id_df["player_id"].nunique()}')

Player name nunique - 3910
Player ID nunique - 3873


**There shouldn't be a mismatch in IDs and names, each ID should have one name, look into further**

In [28]:
# Check for missing values
id_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14209 entries, 1 to 42818
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   player_name  14209 non-null  object
 1   player_id    14209 non-null  object
 2   position     14209 non-null  object
 3   team         14209 non-null  object
 4   std_pos      14209 non-null  object
dtypes: object(5)
memory usage: 666.0+ KB


No missing values, look into why there is name/ID mistmatch

In [29]:
# Group by player_id and count unique player_names
ids_with_multiple_names = (id_df.groupby('player_id')['player_name']
                           .nunique()
                           .reset_index())

# Filter to only those with more than one name
ids_with_multiple_names = ids_with_multiple_names[ids_with_multiple_names['player_name'] > 1]
ids_with_multiple_names

,player_id,player_name
78,00-0020245,2
86,00-0020397,2
460,00-0025944,2
538,00-0026293,2
717,00-0027253,2
...,...,...
3393,00-0038134,2
3416,00-0038409,2
3476,00-0038611,2
3574,00-0038920,2


**Some IDs have multiple names attached to them**

In [30]:
# Take a look three of the IDs to see what the issue is
display(id_df[id_df['player_id'] == '00-0020245'])
display(id_df[id_df['player_id'] == '00-0020397'])
display(id_df[id_df['player_id'] == '00-0025944'])

,player_name,player_id,position,team,std_pos
237,Michael Vick,00-0020245,QB,PHI,QB
2257,Michael Vick,00-0020245,QB,PHI,QB
4351,Michael Vick,00-0020245,QB,PHI,QB
6404,Michael Vick,00-0020245,QB,PHI,QB
8500,Michael Vick,00-0020245,QB,PHI,QB
10624,Michael Vick,00-0020245,QB,NYJ,QB
12775,Michael Vick,00-0020245,QB,PIT,QB
14962,Mike Vick,00-0020245,QB,PIT,QB


,player_name,player_id,position,team,std_pos
259,Chad Ochocinco,00-0020397,WR,CIN,WR
2275,Chad Johnson,00-0020397,WR,CIN,WR
4359,Chad Johnson,00-0020397,WR,NE,WR


,player_name,player_id,position,team,std_pos
1485,Steven Hauschka,00-0025944,K,BLT,K
1486,Steven Hauschka,00-0025944,K,ATL,K
3304,Steven Hauschka,00-0025944,K,DEN,K
5176,Steven Hauschka,00-0025944,K,SEA,K
7025,Steven Hauschka,00-0025944,K,SEA,K
8968,Steven Hauschka,00-0025944,K,SEA,K
10973,Steven Hauschka,00-0025944,K,SEA,K
13028,Steven Hauschka,00-0025944,K,SEA,K
15180,Stephen Hauschka,00-0025944,K,SEA,K
18141,Stephen Hauschka,00-0025944,K,BUF,K


**The user IDs with multiple names appear to be from names not being standardized. Will take the first instance in each repeat and apply to that ID**

There is no correct variation of the player name (i.e Stephen and Steven for the same player are no different) so taking the first occurence and applying to all standardizes

In [31]:
# Group by ID and take the first instance sorted player name
df_grouped = id_df.sort_values(by='player_name').groupby('player_id')[['player_name', 'std_pos', 'team']].first()

#Turn back into dataframe
df_grouped = df_grouped.reset_index()


In [32]:
# Check shape to make sure same # of unique IDs exist
df_grouped.shape

(3873, 4)

In [33]:
# Second work check
display(df_grouped[df_grouped['player_id'] == '00-0020245'])
display(df_grouped[df_grouped['player_id'] == '00-0020397'])
display(df_grouped[df_grouped['player_id'] == '00-0038920'])

,player_id,player_name,std_pos,team
78,00-0020245,Michael Vick,QB,PHI


,player_id,player_name,std_pos,team
86,00-0020397,Chad Johnson,WR,NE


,player_id,player_name,std_pos,team
3574,00-0038920,John Samuel Shenker,TE,LV


In [34]:
# Third work check

# Group by player_id and count unique player_names
ids_with_multiple_names = (df_grouped.groupby('player_id')['player_name']
                           .nunique()
                           .reset_index())

# Filter to only those with more than one name
ids_with_multiple_names = ids_with_multiple_names[ids_with_multiple_names['player_name'] > 1]
ids_with_multiple_names

,player_id,player_name


**Now to check for names that repeat with different IDs**

In [35]:
# Group by player_name and count unique player_ids
names_with_multiple_ids = (id_df.groupby('player_name')['player_id']
                           .nunique()
                           .reset_index())

# Filter to only those with more than one ID
names_with_multiple_ids = names_with_multiple_ids[names_with_multiple_ids['player_id'] > 1]
names_with_multiple_ids

,player_name,player_id
53,Adrian Peterson,2
92,Alex Smith,2
128,Andre Davis,2
220,Austin Allen,2
655,Chris Brooks,2
656,Chris Brown,2
670,Chris Harper,2
671,Chris Henry,2
701,Chris Thompson,2
827,D.J. Williams,2


**Now there are players with the same name nut different IDs. Look into further**

In [36]:
# Check first repeat name on list
rosters_seas_ff[rosters_seas_ff['player_name'] == 'Adrian Peterson'].head()

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
409,2009,CHI,RB,None,29,ACT,Adrian Peterson,Adrian,Peterson,1979-07-01,70.0,212.0,None,00-0021306,None,None,None,None,None,None,None,None,7.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Adrian,PET263012,None,32005045-5426-3012-fd67-09573cdb8335,2002.0,2002.0,CHI,199,30.0
1226,2009,MIN,RB,None,28,ACT,Adrian Peterson,Adrian,Peterson,1985-03-21,73.0,217.0,Oklahoma,00-0025394,10452,ab58c0ac-a747-47e6-9b3c-505e41d2bd3d,8261,5215,3623,PeteAd01,4807,184,2.0,https://static.www.nfl.com/image/private/f_aut...,None,20,CON,A01,Adrian,PET260705,32200,32005045-5426-0705-afaf-b095105b31fb,2007.0,2007.0,MIN,7,24.0
3072,2010,MIN,RB,None,28,ACT,Adrian Peterson,Adrian,Peterson,1985-03-21,73.0,217.0,Oklahoma,00-0025394,10452,ab58c0ac-a747-47e6-9b3c-505e41d2bd3d,8261,5215,3623,PeteAd01,4807,184,3.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Adrian,PET260705,32200,32005045-5426-0705-afaf-b095105b31fb,2007.0,2007.0,MIN,7,25.0
5001,2011,MIN,RB,None,28,RES,Adrian Peterson,Adrian,Peterson,1985-03-21,73.0,217.0,Oklahoma,00-0025394,10452,ab58c0ac-a747-47e6-9b3c-505e41d2bd3d,8261,5215,3623,PeteAd01,4807,184,4.0,https://static.www.nfl.com/image/private/f_aut...,None,16,REG,A01,Adrian,PET260705,32200,32005045-5426-0705-afaf-b095105b31fb,2007.0,2007.0,MIN,7,26.0
6880,2012,MIN,RB,None,28,ACT,Adrian Peterson,Adrian,Peterson,1985-03-21,73.0,217.0,Oklahoma,00-0025394,10452,ab58c0ac-a747-47e6-9b3c-505e41d2bd3d,8261,5215,3623,PeteAd01,4807,184,5.0,https://static.www.nfl.com/image/private/f_aut...,None,18,WC,A01,Adrian,PET260705,32200,32005045-5426-0705-afaf-b095105b31fb,2007.0,2007.0,MIN,7,27.0


**Above shows two different players with same name and two different IDs/stat sets**

In [37]:
# Check second repeat name on list
rosters_seas_ff[rosters_seas_ff['player_name'] == 'Alex Smith'].head()

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
779,2009,SF,QB,None,11,ACT,Alex Smith,Alexander,Smith,1984-05-07,76.0,217.0,Utah,00-0023436,8416,2fda010a-8c62-4c07-b601-4ba03f57e6af,7177,4306,2218,SmitAl03,6739,268,4.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Alex,SMI031126,29819,3200534d-4903-1126-6414-eeba5603cc44,2005.0,2005.0,SF,1,25.0
841,2009,PHI,TE,None,82,ACT,Alex Smith,Edwin,Smith,1982-05-22,76.0,258.0,Stanford,00-0023506,None,None,None,None,None,None,None,None,4.0,https://static.www.nfl.com/image/private/f_aut...,None,18,WC,A01,Alex,SMI029398,29898,3200534d-4902-9398-d7ad-d21cbbe6a53d,2005.0,2005.0,TB,71,27.0
2692,2010,SF,QB,None,11,ACT,Alex Smith,Alexander,Smith,1984-05-07,76.0,217.0,Utah,00-0023436,8416,2fda010a-8c62-4c07-b601-4ba03f57e6af,7177,4306,2218,SmitAl03,6739,268,5.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Alex,SMI031126,29819,3200534d-4903-1126-6414-eeba5603cc44,2005.0,2005.0,SF,1,26.0
2746,2010,CLV,TE,None,81,ACT,Alex Smith,Edwin,Smith,1982-05-22,76.0,258.0,Stanford,00-0023506,None,None,None,None,None,None,None,None,5.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Alex,SMI029398,29898,3200534d-4902-9398-d7ad-d21cbbe6a53d,2005.0,2005.0,TB,71,28.0
4689,2011,SF,QB,None,11,ACT,Alex Smith,Alexander,Smith,1984-05-07,76.0,217.0,Utah,00-0023436,8416,2fda010a-8c62-4c07-b601-4ba03f57e6af,7177,4306,2218,SmitAl03,6739,268,6.0,https://static.www.nfl.com/image/private/f_aut...,None,20,CON,A01,Alex,SMI031126,29819,3200534d-4903-1126-6414-eeba5603cc44,2005.0,2005.0,SF,1,27.0


**This name is also a repeat name but two separate players**

Since I am creating a unique ID column with player name/team/season/position, these name duplicates will be handled with the unique ID I create. No changes needed. Just need to map cleaned names to datasets

In [38]:
# Retitle player naem column to cleaned naem for specificity
df_grouped = df_grouped.rename(columns={'player_name': 'cleaned_name'})

In [39]:
df_grouped

,player_id,cleaned_name,std_pos,team
0,00-0000108,David Akers,K,PHI
1,00-0000865,Charlie Batch,QB,PIT
2,00-0001477,Marty Booker,WR,ATL
3,00-0001545,Todd Bouman,QB,JAX
4,00-0001980,Kris Brown,K,HST
...,...,...,...,...
3868,00-0039940,Kairee Robinson,RB,SEA
3869,00-0039941,Geor'Quarius Spivey,TE,KC
3870,00-0039942,Mason Fairchild,TE,NO
3871,00-0039945,Alex Hale,K,GB


In [40]:
#Create set with cleaned name and player ID as index for dicitonary mapping
grouped_name_id = df_grouped.set_index('player_id').drop(columns=['std_pos', 'team'])
grouped_name_id.head()

,cleaned_name
player_id,
00-0000108,David Akers
00-0000865,Charlie Batch
00-0001477,Marty Booker
00-0001545,Todd Bouman
00-0001980,Kris Brown


In [41]:
# Create dictionary for mapping
df_dict = grouped_name_id.to_dict()['cleaned_name']
df_dict

{'00-0000108': 'David Akers',
 '00-0000865': 'Charlie Batch',
 '00-0001477': 'Marty Booker',
 '00-0001545': 'Todd Bouman',
 '00-0001980': 'Kris Brown',
 '00-0002099': 'Isaac Bruce',
 '00-0002110': 'Mark Brunell',
 '00-0002655': 'John Carney',
 '00-0003035': 'Desmond Clark',
 '00-0003292': 'Kerry Collins',
 '00-0003315': 'Todd Collins',
 '00-0003739': 'Daunte Culpepper',
 '00-0004091': 'Phil Dawson',
 '00-0004161': 'Jake Delhomme',
 '00-0004541': 'Donald Driver',
 '00-0004811': 'Jason Elam',
 '00-0004915': 'Bobby Engram',
 '00-0005091': 'Kevin Faulk',
 '00-0005106': 'Brett Favre',
 '00-0005231': 'Brian Finneran',
 '00-0005720': 'Joey Galloway',
 '00-0005755': 'Jeff Garcia',
 '00-0006101': 'Tony Gonzalez',
 '00-0006305': 'Ahman Green',
 '00-0006410': 'Chris Greisen',
 '00-0006800': 'Jason Hanson',
 '00-0007091': 'Matt Hasselbeck',
 '00-0007213': 'Steve Heiden',
 '00-0007681': 'Torry Holt',
 '00-0008241': 'Edgerrin James',
 '00-0009028': 'John Kasay',
 '00-0009119': 'Reggie Kelly',
 '00-0

In [42]:
# Double check position dictionary
pos_dict

{'00-0000108': 'K',
 '00-0000865': 'QB',
 '00-0001477': 'WR',
 '00-0001545': 'QB',
 '00-0001980': 'K',
 '00-0002099': 'WR',
 '00-0002110': 'QB',
 '00-0002655': 'K',
 '00-0003035': 'TE',
 '00-0003292': 'QB',
 '00-0003315': 'QB',
 '00-0003739': 'QB',
 '00-0004091': 'K',
 '00-0004161': 'QB',
 '00-0004541': 'WR',
 '00-0004811': 'K',
 '00-0004915': 'WR',
 '00-0005091': 'RB',
 '00-0005106': 'QB',
 '00-0005231': 'WR',
 '00-0005720': 'WR',
 '00-0005755': 'QB',
 '00-0006101': 'TE',
 '00-0006305': 'RB',
 '00-0006410': 'QB',
 '00-0006800': 'K',
 '00-0007091': 'QB',
 '00-0007213': 'TE',
 '00-0007681': 'WR',
 '00-0008241': 'RB',
 '00-0009028': 'K',
 '00-0009119': 'TE',
 '00-0009311': 'QB',
 '00-0009323': 'TE',
 '00-0010072': 'K',
 '00-0010346': 'QB',
 '00-0010373': 'K',
 '00-0010500': 'WR',
 '00-0011022': 'QB',
 '00-0011641': 'WR',
 '00-0011754': 'WR',
 '00-0011795': 'WR',
 '00-0011970': 'K',
 '00-0012478': 'WR',
 '00-0013694': 'RB',
 '00-0014655': 'RB',
 '00-0015601': 'RB',
 '00-0015754': 'WR',
 '

**Map standardized names and positions to datasets**

**Weekly rosters**

In [44]:
# Make copy of weekly fantasy players
rw_clean = rosters_weekly_ff.copy()

# Map cleaned names to IDs
rw_clean['cleaned_name'] = rw_clean['player_id'].map(df_dict)

# Map standard positions to team names
rw_clean['std_pos'] = rw_clean['player_id'].map(pos_dict)

# Data check
rw_clean

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age,cleaned_name,std_pos
19,2009,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,12.0,https://static.www.nfl.com/image/private/f_aut...,None,7,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,34.880,David Akers,K
20,2009,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,12.0,https://static.www.nfl.com/image/private/f_aut...,None,2,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,34.782,David Akers,K
21,2009,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,12.0,https://static.www.nfl.com/image/private/f_aut...,None,4,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,NaN,David Akers,K
22,2009,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,12.0,https://static.www.nfl.com/image/private/f_aut...,None,5,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,34.839,David Akers,K
23,2009,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,12.0,https://static.www.nfl.com/image/private/f_aut...,None,6,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,34.858,David Akers,K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641678,2024,GB,K,K,16.0,DEV,Alex Hale,Alex,Hale,1998-03-07,72.0,205.0,Oklahoma State,00-0039945,4361831,c0f5daf0-5505-11ef-b7e3-6370a6e73976,None,18429,None,None,None,12438,0.0,None,None,8,REG,P03,Alex,HAL031251,58041,32004841-4c03-1251-97e3-c8a0c955a05c,2024.0,2024.0,None,NaN,26.642,Alex Hale,K
641679,2024,GB,K,K,16.0,DEV,Alex Hale,Alex,Hale,1998-03-07,72.0,205.0,Oklahoma State,00-0039945,4361831,c0f5daf0-5505-11ef-b7e3-6370a6e73976,None,18429,None,None,None,12438,0.0,None,None,15,REG,P03,Alex,HAL031251,58041,32004841-4c03-1251-97e3-c8a0c955a05c,2024.0,2024.0,None,NaN,26.776,Alex Hale,K
641680,2024,GB,K,K,16.0,DEV,Alex Hale,Alex,Hale,1998-03-07,72.0,205.0,Oklahoma State,00-0039945,4361831,c0f5daf0-5505-11ef-b7e3-6370a6e73976,None,18429,None,None,None,12438,0.0,None,None,14,REG,P03,Alex,HAL031251,58041,32004841-4c03-1251-97e3-c8a0c955a05c,2024.0,2024.0,None,NaN,26.749,Alex Hale,K
641681,2024,GB,K,K,16.0,DEV,Alex Hale,Alex,Hale,1998-03-07,72.0,205.0,Oklahoma State,00-0039945,4361831,c0f5daf0-5505-11ef-b7e3-6370a6e73976,None,18429,None,None,None,12438,0.0,None,None,19,WC,P03,Alex,HAL031251,58041,32004841-4c03-1251-97e3-c8a0c955a05c,2024.0,2024.0,None,NaN,26.853,Alex Hale,K


In [45]:
# Check that all IDs exist still
rw_clean['player_id'].nunique()

3877

**Seasonal Rosters**

In [46]:
# Make copy of seasonal fantasy players
rs_clean = rosters_seas_ff.copy()

# Map cleaned names to IDs
rs_clean['cleaned_name'] = rs_clean['player_id'].map(df_dict)

# Map standard positions to team names
rs_clean['std_pos'] = rs_clean['player_id'].map(pos_dict)

# Data check
rs_clean

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age,cleaned_name,std_pos
1,2009,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,12.0,https://static.www.nfl.com/image/private/f_aut...,None,18,WC,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,34.0,David Akers,K
7,2009,PIT,QB,None,16,ACT,Charlie Batch,Charles,Batch,1974-12-05,74.0,216.0,None,00-0000865,1490,a725e7c5-86df-4b5b-abe0-71b809be988d,None,945,367,BatcCh00,None,None,11.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,I02,Charlie,BAT039161,None,32004241-5403-9161-d21e-19cbefc1c6b0,1998.0,1998.0,DET,60,34.0,Charlie Batch,QB
13,2009,ATL,WR,None,80,ACT,Marty Booker,Marty,Booker,1976-07-31,72.0,205.0,None,00-0001477,None,None,None,None,None,None,None,None,10.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Marty,BOO033965,None,3200424f-4f03-3965-c6e4-7a66e7da27ec,1999.0,1999.0,CHI,78,33.0,Marty Booker,WR
16,2009,HST,K,None,3,ACT,Kris Brown,Kristopher,Brown,1976-12-23,71.0,212.0,None,00-0001980,None,None,None,None,None,None,None,None,10.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Kris,BRO521985,None,32004252-4f52-1985-bb20-5663ab81e524,1999.0,1999.0,PIT,228,32.0,Kris Brown,K
17,2009,SF,WR,None,88,ACT,Isaac Bruce,Isaac,Bruce,1972-11-10,72.0,188.0,None,00-0002099,None,None,None,None,None,None,None,None,15.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,Isaac,BRU085284,None,32004252-5508-5284-f394-5e776b7d5d81,1994.0,1994.0,LA,33,36.0,Isaac Bruce,WR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42806,2024,SEA,RB,RB,35.0,CUT,Kairee Robinson,Kairee,Robinson,2000-02-23,67.0,195.0,San Jose State,00-0039940,None,None,None,None,None,None,None,None,0.0,None,None,1,REG,W03,Kairee,ROB591051,58030,3200524f-4259-1051-9b5d-a2fec8e616e8,2024.0,2024.0,None,NaN,24.0,Kairee Robinson,RB
42807,2024,KC,TE,TE,85.0,CUT,Geor'Quarius Spivey,Geor'quarius,Spivey,1999-10-06,77.0,240.0,Mississippi State,00-0039941,None,None,None,None,None,None,None,None,0.0,None,None,1,REG,W03,Geor'quarius,SPI709879,58035,32005350-4970-9879-b7b8-8108e0dbca38,2024.0,2024.0,None,NaN,24.0,Geor'Quarius Spivey,TE
42808,2024,NO,TE,TE,82.0,CUT,Mason Fairchild,Mason,Fairchild,2001-08-30,76.0,260.0,Kansas,00-0039942,None,None,None,None,None,None,None,None,0.0,None,None,1,REG,P01,Mason,FAI339781,58004,32004641-4933-9781-9d27-8d73b06581dc,2024.0,2024.0,None,NaN,23.0,Mason Fairchild,TE
42811,2024,GB,K,K,16.0,DEV,Alex Hale,Alex,Hale,1998-03-07,72.0,205.0,Oklahoma State,00-0039945,4361831,c0f5daf0-5505-11ef-b7e3-6370a6e73976,None,18429,None,None,None,12438,0.0,None,None,19,WC,P03,Alex,HAL031251,58041,32004841-4c03-1251-97e3-c8a0c955a05c,2024.0,2024.0,None,NaN,26.0,Alex Hale,K


In [47]:
# Check that all IDs exist still
rs_clean['player_id'].nunique()

3873

**Seasonal Stats**

In [48]:
# Make copy of seasonal statistics
ss_clean = stats_seas.copy()

# Map cleaned names to IDs
ss_clean['cleaned_name'] = ss_clean['player_id'].map(df_dict)

# Map standard positions to team names
ss_clean['std_pos'] = ss_clean['player_id'].map(pos_dict)

# Data Check
ss_clean

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,cleaned_name,std_pos
0,00-0000108,2011,REG,1,1,14.0,1,0.0,0.0,0.0,0,0,9.0,5.0,1.0,4.223134,0,1.555556,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,4.56,4.56,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.047313,David Akers,K
1,00-0000741,2009,REG,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,1.0,6.00,6.00,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.067568,NaN,NaN
2,00-0000865,2009,REG,1,2,17.0,0,0.0,0.0,0.0,0,0,18.0,2.0,1.0,1.449849,0,0.944444,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,0.68,0.68,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.005375,Charlie Batch,QB
3,00-0000865,2010,REG,29,49,352.0,3,3.0,4.0,21.0,0,0,538.0,125.0,13.0,-2.543351,0,1.956525,0.381428,7,30.0,0,1.0,0.0,2.0,1.690736,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,23.08,23.08,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.135845,Charlie Batch,QB
4,00-0000865,2011,REG,15,24,208.0,0,1.0,2.0,10.0,0,0,189.0,92.0,9.0,-0.096867,0,1.149171,0.154437,3,-2.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,6.12,6.12,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.024270,Charlie Batch,QB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11739,00-0039917,2024,REG,61,105,775.0,3,3.0,4.0,14.0,2,0,1073.0,271.0,35.0,14.553117,0,7.686124,0.250735,7,11.0,1,0.0,0.0,1.0,0.585619,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,44.10,44.10,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.107378,Michael Penix Jr.,QB
11740,00-0039918,2024,REG,351,562,3541.0,20,6.0,68.0,466.0,7,3,4486.0,1866.0,171.0,-43.804843,3,15.246817,1.290656,81,489.0,0,3.0,2.0,27.0,11.632084,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,254.54,254.54,17,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.196150,Caleb Williams,QB
11741,00-0039919,2024,REG,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,3,15.0,0,0.0,0.0,1.0,0.575859,0,54,101,734.0,3,2.0,1.0,1398.0,253.0,37.0,16.061161,0,10.960008,3.215179,5.542503,8.702520,0.0,90.90,144.90,17,0.178445,0.308337,0.134861,0.514337,0.206644,0.15,0.215116,0.208333,0.178322,0.195315,1.296820,0.111661,Rome Odunze,WR
11742,00-0039920,2024,REG,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,2,26.0,0,1.0,0.0,1.0,-4.431244,0,3,6,16.0,0,0.0,0.0,50.0,5.0,1.0,-3.643815,0,1.166667,0.182203,0.225307,0.431020,0.0,

In [49]:
# Make copy of seasonal statistics
ss_clean = stats_seas.copy()

# Map cleaned names to IDs
ss_clean['cleaned_name'] = ss_clean['player_id'].map(df_dict)

# Map standard positions to team names
ss_clean['std_pos'] = ss_clean['player_id'].map(pos_dict)

# Data Check
ss_clean

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,cleaned_name,std_pos
0,00-0000108,2011,REG,1,1,14.0,1,0.0,0.0,0.0,0,0,9.0,5.0,1.0,4.223134,0,1.555556,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,4.56,4.56,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.047313,David Akers,K
1,00-0000741,2009,REG,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,1.0,6.00,6.00,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.067568,NaN,NaN
2,00-0000865,2009,REG,1,2,17.0,0,0.0,0.0,0.0,0,0,18.0,2.0,1.0,1.449849,0,0.944444,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,0.68,0.68,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.005375,Charlie Batch,QB
3,00-0000865,2010,REG,29,49,352.0,3,3.0,4.0,21.0,0,0,538.0,125.0,13.0,-2.543351,0,1.956525,0.381428,7,30.0,0,1.0,0.0,2.0,1.690736,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,23.08,23.08,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.135845,Charlie Batch,QB
4,00-0000865,2011,REG,15,24,208.0,0,1.0,2.0,10.0,0,0,189.0,92.0,9.0,-0.096867,0,1.149171,0.154437,3,-2.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,6.12,6.12,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.024270,Charlie Batch,QB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11739,00-0039917,2024,REG,61,105,775.0,3,3.0,4.0,14.0,2,0,1073.0,271.0,35.0,14.553117,0,7.686124,0.250735,7,11.0,1,0.0,0.0,1.0,0.585619,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,44.10,44.10,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.107378,Michael Penix Jr.,QB
11740,00-0039918,2024,REG,351,562,3541.0,20,6.0,68.0,466.0,7,3,4486.0,1866.0,171.0,-43.804843,3,15.246817,1.290656,81,489.0,0,3.0,2.0,27.0,11.632084,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,254.54,254.54,17,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.196150,Caleb Williams,QB
11741,00-0039919,2024,REG,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,3,15.0,0,0.0,0.0,1.0,0.575859,0,54,101,734.0,3,2.0,1.0,1398.0,253.0,37.0,16.061161,0,10.960008,3.215179,5.542503,8.702520,0.0,90.90,144.90,17,0.178445,0.308337,0.134861,0.514337,0.206644,0.15,0.215116,0.208333,0.178322,0.195315,1.296820,0.111661,Rome Odunze,WR
11742,00-0039920,2024,REG,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,2,26.0,0,1.0,0.0,1.0,-4.431244,0,3,6,16.0,0,0.0,0.0,50.0,5.0,1.0,-3.643815,0,1.166667,0.182203,0.225307,0.431020,0.0,

**Check for NaNs in cleaned name -- this means the ID is in the statistics but wasn't in the other sets**

In [50]:
# Check how many naans are present
na_names = ss_clean[ss_clean['cleaned_name'].isna()]

# Check shape for number missing
print(f'missing stats names shape - {na_names.shape}')

# Look at stat layout for those with a missing name
na_names.describe()

missing stats names shape - (619, 60)


,season,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
count,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.0,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.0,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.00000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,524.000000,619.000000,619.000000,524.000000,524.000000,619.000000,619.000000
mean,2016.423263,0.147011,0.232633,2.520194,0.017771,0.008078,0.009693,0.072698,0.008078,0.0,2.084006,1.360258,0.127625,0.188127,0.004847,0.283000,0.0,0.423263,1.946688,0.017771,0.101777,0.024233,0.176090,-0.155647,0.003231,0.263328,0.387722,2.130856,0.090468,0.008078,0.006462,1.279483,1.525040,0.17609,0.080316,0.006462,0.383791,0.011729,0.004302,0.020605,0.200323,2.382553,2.645880,1.287561,0.009284,0.003541,0.010538,0.016759,0.006872,0.048705,0.012522,0.016535,0.027938,0.015478,0.049972,0.026951
std,4.550526,0.405508,0.561005,8.121897,0.132223,0.089584,0.098054,0.949986,0.089584,0.0,6.840725,5.090232,0.379315,1.528116,0.069504,1.126089,0.0,0.654960,8.843409,0.143942,0.318238,0.153895,0.447586,2.122372,0.056796,0.607501,0.776836,8.506461,0.333981,0.089584,0.080191,5.680354,6.425873,0.47902,1.420828,0.080191,1.285638,0.024428,0.019895,0.045957,0.442777,3.534058,3.802758,0.825184,0.014247,0.015001,0.038631,0.027961,0.026463,0.167537,0.031712,0.044565,0.088508,0.044444,0.194494,0.035848
min,2009.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-8.000000,0.000000,0.000000,-5.712035,0.000000,0.000000,0.0,0.000000,-28.000000,0.000000,0.000000,0.000000,0.000000,-9.848605,0.000000,0.000000,0.000000,-11.000000,0.000000,0.000000,0.000000,-17.000000,-4.000000,0.00000,-6.566578,0.000000,0.000000,0.000000,-0.086538,0.000000,0.000000,-3.800000,-3.800000,1.000000,0.000000,-0.078947,-0.021978,-0.002632,-0.058201,0.000000,0.000000,0.000000,-0.029101,-0.046561,-0.289474,-0.037322
25%,2012.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2016.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.760000,1.060000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011297
75%,2020.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00

**The average for all the stats above are very low, seems like they may not be fantasy relevant players. Since no names or position existed in season stats to begin with, the fantasy relevant filter was not applied**

Look into whether or not the missing names are fantasy players. The season rosters dataset included all positons

In [51]:
# Make df with players from the seasonal roster that dont have names in the stats roster
miss_names = rosters_seas[rosters_seas['player_id'].isin(na_names['player_id'])]

# Look at positions values in the missing names set
miss_names['position'].value_counts()

DB     670
P      502
OL     441
LB     193
T      163
FS     154
CB     147
SS     120
DL      96
G       81
OLB     59
DE      51
C       44
DT      25
LS      23
NT      16
MLB     10
ILB      8
S        1
Name: position, dtype: int64

**All missing names from the stats set are non-fantasy relevent positions. Will switch merge to inner to keep only overlaps**

In [52]:
# Drop nas in cleaned name, they are not fantasy relevant players
ss_clean = ss_clean.dropna(subset=['cleaned_name'])

# Check at how many play IDs remain
ss_clean['player_id'].nunique()

2436

**I will be agggregating weekly data down to seasonal. The players ID will stay the same but their team could be different year in and year out**

I will create a team/player/season mapping ot mak sure the correct team name is applied to the correct player and season

In [53]:
# Create team column that lists team for that player for that season
team_grouped = rs_clean.groupby(['player_id', 'season'])[['cleaned_name', 'team']].first().reset_index()
team_grouped

,player_id,season,cleaned_name,team
0,00-0000108,2009,David Akers,PHI
1,00-0000108,2010,David Akers,PHI
2,00-0000108,2011,David Akers,SF
3,00-0000108,2012,David Akers,SF
4,00-0000108,2013,David Akers,DET
...,...,...,...,...
13947,00-0039940,2024,Kairee Robinson,SEA
13948,00-0039941,2024,Geor'Quarius Spivey,KC
13949,00-0039942,2024,Mason Fairchild,NO
13950,00-0039945,2024,Alex Hale,GB


Consolidate down to one ID per player/season for proper mapping

In [54]:
# Create ID/Season column
team_grouped['id_season'] = team_grouped['player_id'] + '_' + team_grouped['season'].astype(str)

#Export to CSV to visual inspection
team_grouped.to_csv('teams_and_ids.csv')


**Upon visual inspection, teams were allocated per player/season correctly**

In [55]:
# Keep only columns with team and id/season combo
team_grouped = team_grouped.drop(columns=['player_id', 'season', 'cleaned_name']).set_index('id_season')
team_grouped

,team
id_season,
00-0000108_2009,PHI
00-0000108_2010,PHI
00-0000108_2011,SF
00-0000108_2012,SF
00-0000108_2013,DET
...,...
00-0039940_2024,SEA
00-0039941_2024,KC
00-0039942_2024,NO


In [58]:
# Create dictionary for team mapping
team_dict = team_grouped.to_dict()['team']
team_dict

{'00-0000108_2009': 'PHI',
 '00-0000108_2010': 'PHI',
 '00-0000108_2011': 'SF',
 '00-0000108_2012': 'SF',
 '00-0000108_2013': 'DET',
 '00-0000865_2009': 'PIT',
 '00-0000865_2010': 'PIT',
 '00-0000865_2011': 'PIT',
 '00-0000865_2012': 'PIT',
 '00-0001477_2009': 'ATL',
 '00-0001545_2010': 'JAX',
 '00-0001980_2009': 'HST',
 '00-0001980_2010': 'DAL',
 '00-0002099_2009': 'SF',
 '00-0002110_2009': 'NO',
 '00-0002110_2010': 'NYJ',
 '00-0002110_2011': 'NYJ',
 '00-0002655_2009': 'NO',
 '00-0002655_2010': 'NO',
 '00-0003035_2009': 'CHI',
 '00-0003035_2010': 'CHI',
 '00-0003292_2009': 'TEN',
 '00-0003292_2010': 'TEN',
 '00-0003292_2011': 'IND',
 '00-0003315_2009': 'WAS',
 '00-0003315_2010': 'CHI',
 '00-0003739_2009': 'DET',
 '00-0004091_2009': 'CLV',
 '00-0004091_2010': 'CLV',
 '00-0004091_2011': 'CLV',
 '00-0004091_2012': 'CLV',
 '00-0004091_2013': 'SF',
 '00-0004091_2014': 'SF',
 '00-0004091_2015': 'SF',
 '00-0004091_2016': 'SF',
 '00-0004091_2017': 'ARI',
 '00-0004091_2018': 'ARI',
 '00-000416

In [61]:
# Create ID season column for mapping
ss_clean['id_season'] = ss_clean['player_id'] + '_' + ss_clean['season'].astype(str)

# Map teams to season
ss_clean['team'] = ss_clean['id_season'].map(team_dict)

# Head Check
ss_clean.head()

C:\Users\nick_\AppData\Local\Temp\ipykernel_22324\2581320385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss_clean['id_season'] = ss_clean['player_id'] + '_' + ss_clean['season'].astype(str)
C:\Users\nick_\AppData\Local\Temp\ipykernel_22324\2581320385.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss_clean['team'] = ss_clean['id_season'].map(team_dict)


,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,cleaned_name,std_pos,id_season,team
0,00-0000108,2011,REG,1,1,14.0,1,0.0,0.0,0.0,0,0,9.0,5.0,1.0,4.223134,0,1.555556,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.56,4.56,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047313,David Akers,K,00-0000108_2011,SF
2,00-0000865,2009,REG,1,2,17.0,0,0.0,0.0,0.0,0,0,18.0,2.0,1.0,1.449849,0,0.944444,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.68,0.68,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005375,Charlie Batch,QB,00-0000865_2009,PIT
3,00-0000865,2010,REG,29,49,352.0,3,3.0,4.0,21.0,0,0,538.0,125.0,13.0,-2.543351,0,1.956525,0.381428,7,30.0,0,1.0,0.0,2.0,1.690736,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,23.08,23.08,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.135845,Charlie Batch,QB,00-0000865_2010,PIT
4,00-0000865,2011,REG,15,24,208.0,0,1.0,2.0,10.0,0,0,189.0,92.0,9.0,-0.096867,0,1.149171,0.154437,3,-2.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,6.12,6.12,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024270,Charlie Batch,QB,00-0000865_2011,PIT
5,00-0000865,2012,REG,45,70,475.0,1,4.0,3.0,12.0,1,0,663.0,257.0,22.0,-1.646032,0,1.429991,0.160617,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,15.00,15.00,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.119048,Charlie Batch,QB,00-0000865_2012,PIT


**Injuries have a different unique ID than the rest of the sets. This poses a challenge for merge, look into further**

In [62]:
injuries.head()

,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
0,2009.0,REG,ARI,1.0,00-0022084,WR,Anquan Boldin,Anquan,Boldin,Hamstring,None,Questionable,Hamstring,None,Limited Participation in Practice,NaT
1,2009.0,REG,ARI,1.0,00-0026221,WR,Early Doucet,Early,Doucet,Ribs,None,Questionable,Ribs,None,Limited Participation in Practice,NaT
2,2009.0,REG,ARI,1.0,00-0022101,QB,Brian St. Pierre,Brian,St. Pierre,Back,None,Questionable,Back,None,Limited Participation in Practice,NaT
3,2009.0,REG,ARI,1.0,00-0025529,WR,Steve Breaston,Steve,Breaston,Knee,None,Probable,Knee,None,Full Participation in Practice,NaT
4,2009.0,REG,ARI,1.0,00-0022786,S,Matt Ware,Matt,Ware,Shoulder,None,Probable,Shoulder,None,Full Participation in Practice,NaT


**Check for ids that are in the seasonal dataset but not the injury dataset**

They may actually be the same number but with a different column label. Can retitle if that ends up being the case

In [63]:
# Get unique player IDs from each dataframe
injuries_ids = injuries_ff['gsis_id'].unique()
season_ids = rs_clean['player_id'].unique()

injuries_ids_set = set(injuries_ids)
season_ids_set = set(season_ids)

# Find IDs in weekly but not in season
ids_in_season_no_injuries = season_ids_set - injuries_ids_set

# Convert the result to a list
ids_list = list(ids_in_season_no_injuries)

print(f"Number of player IDs in rs_clean but not in injuries: {len(ids_list)}")

# Create a dataframe with these IDs and their details from rosters_weekly
missing_players_df = rs_clean[rs_clean['player_id'].isin(ids_list)]

# To get a unique list of these players (without duplicates)
unique_missing_players_df = missing_players_df.drop_duplicates(subset=['player_id'])

# Display the resulting dataframe
print(unique_missing_players_df[['player_id', 'player_name', 'position', 'status']])

Number of player IDs in rs_clean but not in injuries: 1763
        player_id          player_name position status
23     00-0003739     Daunte Culpepper       QB    ACT
35     00-0004915         Bobby Engram       WR    CUT
45     00-0005720        Joey Galloway       WR    ACT
47     00-0005755          Jeff Garcia       QB    CUT
62     00-0008241       Edgerrin James       RB    CUT
...           ...                  ...      ...    ...
42806  00-0039940      Kairee Robinson       RB    CUT
42807  00-0039941  Geor'Quarius Spivey       TE    CUT
42808  00-0039942      Mason Fairchild       TE    CUT
42811  00-0039945            Alex Hale        K    DEV
42818  00-0039952       Devon Garrison       TE    CUT

[1763 rows x 4 columns]


In [66]:
# Check for one of the missing names in the injuries sheet to see if it is there
display(injuries_ff[injuries_ff['full_name'] == 'Daunte Culpepper'])
display(injuries_ff[injuries_ff['full_name'] == 'Joey Galloway'])

,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified


,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified


**From a quick check, the missing names from the injury list are players who didn't have any injuries in the period**

This means the remaining IDs are present, aka the gsis ID in injuries and the player ID in other datasets are the same. Will retitle

In [69]:
# Retitle gsis column
injuries_ff = injuries_ff.rename(columns={'gsis_id' : 'player_id'})

# Head Check
injuries_ff.head()

,season,game_type,team,week,player_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
0,2009.0,REG,ARI,1.0,00-0022084,WR,Anquan Boldin,Anquan,Boldin,Hamstring,None,Questionable,Hamstring,None,Limited Participation in Practice,NaT
1,2009.0,REG,ARI,1.0,00-0026221,WR,Early Doucet,Early,Doucet,Ribs,None,Questionable,Ribs,None,Limited Participation in Practice,NaT
2,2009.0,REG,ARI,1.0,00-0022101,QB,Brian St. Pierre,Brian,St. Pierre,Back,None,Questionable,Back,None,Limited Participation in Practice,NaT
3,2009.0,REG,ARI,1.0,00-0025529,WR,Steve Breaston,Steve,Breaston,Knee,None,Probable,Knee,None,Full Participation in Practice,NaT
5,2009.0,REG,ARI,1.0,00-0011641,WR,Sean Morey,Sean,Morey,Ribs,None,Probable,Ribs,None,Limited Participation in Practice,NaT


In [ ]:
name_check = injuries_ff['full_name'].head().tolist()

In [ ]:
df_grouped[df_grouped['cleaned_name'].isin(name_check)]

,player_id,cleaned_name,std_pos,team
39,00-0011641,Sean Morey,WR,ARZ
165,00-0022084,Anquan Boldin,WR,BUF
169,00-0022101,Brian St. Pierre,QB,CAR
421,00-0025529,Steve Breaston,WR,KC
518,00-0026221,Early Doucet,WR,ARZ


**Now that player ID is standardized, map info to injuries**

In [70]:
# Make copy of injury statistics
inj_clean = injuries_ff.copy()

# Map cleaned names to IDs
inj_clean['cleaned_name'] = inj_clean['player_id'].map(df_dict)

# Map standard positions to team names
inj_clean['std_pos'] = inj_clean['player_id'].map(pos_dict)

# Data check
inj_clean

,season,game_type,team,week,player_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified,cleaned_name,std_pos
0,2009.0,REG,ARI,1.0,00-0022084,WR,Anquan Boldin,Anquan,Boldin,Hamstring,None,Questionable,Hamstring,None,Limited Participation in Practice,NaT,Anquan Boldin,WR
1,2009.0,REG,ARI,1.0,00-0026221,WR,Early Doucet,Early,Doucet,Ribs,None,Questionable,Ribs,None,Limited Participation in Practice,NaT,Early Doucet,WR
2,2009.0,REG,ARI,1.0,00-0022101,QB,Brian St. Pierre,Brian,St. Pierre,Back,None,Questionable,Back,None,Limited Participation in Practice,NaT,Brian St. Pierre,QB
3,2009.0,REG,ARI,1.0,00-0025529,WR,Steve Breaston,Steve,Breaston,Knee,None,Probable,Knee,None,Full Participation in Practice,NaT,Steve Breaston,WR
5,2009.0,REG,ARI,1.0,00-0011641,WR,Sean Morey,Sean,Morey,Ribs,None,Probable,Ribs,None,Limited Participation in Practice,NaT,Sean Morey,WR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207,2024.0,SB,PHI,22.0,00-0036919,RB,Kenneth Gainwell,Kenneth,Gainwell,None,None,None,Concussion,Knee,Full Participation in Practice,2025-02-07 20:29:05,Kenneth Gainwell,RB
6208,2024.0,SB,PHI,22.0,00-0034351,TE,Dallas Goedert,Dallas,Goedert,None,None,None,Ankle,None,Full Participation in Practice,2025-02-07 20:29:05,Dallas Goedert,TE
6210,2024.0,SB,PHI,22.0,00-0036912,WR,DeVonta Smith,DeVonta,Smith,None,None,None,Hamstring,None,Limited Participation in Practice,2025-02-07 20:29:05,DeVonta Smith,WR
6211,2024.0,SB,PHI,22.0,00-0037132,WR,Britain Covey,Britain,Covey,Neck,None,Out,Neck,None,Full Participation in Practice,2025-02-07 20:32:07,Britain Covey,WR


In [73]:
# Convert season and week to integer for unique ID purposes
inj_clean[['season', 'week']] = inj_clean[['season', 'week']].astype(int)

# Work check
inj_clean.head()

,season,game_type,team,week,player_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified,cleaned_name,std_pos
0,2009,REG,ARI,1,00-0022084,WR,Anquan Boldin,Anquan,Boldin,Hamstring,None,Questionable,Hamstring,None,Limited Participation in Practice,NaT,Anquan Boldin,WR
1,2009,REG,ARI,1,00-0026221,WR,Early Doucet,Early,Doucet,Ribs,None,Questionable,Ribs,None,Limited Participation in Practice,NaT,Early Doucet,WR
2,2009,REG,ARI,1,00-0022101,QB,Brian St. Pierre,Brian,St. Pierre,Back,None,Questionable,Back,None,Limited Participation in Practice,NaT,Brian St. Pierre,QB
3,2009,REG,ARI,1,00-0025529,WR,Steve Breaston,Steve,Breaston,Knee,None,Probable,Knee,None,Full Participation in Practice,NaT,Steve Breaston,WR
5,2009,REG,ARI,1,00-0011641,WR,Sean Morey,Sean,Morey,Ribs,None,Probable,Ribs,None,Limited Participation in Practice,NaT,Sean Morey,WR


**Create Unique ID for each player/season**

In [74]:
# Name of dfs to loop over
cleaned_dfs = [rw_clean, ss_clean, inj_clean]

# Loop to create unique ID column
for df in cleaned_dfs:
  df['unique_id'] = (df['player_id'] + '_' + df['season'].astype(str) + '_' + df['cleaned_name'] + '_' + df['std_pos']).str.replace(' ', '_')

C:\Users\nick_\AppData\Local\Temp\ipykernel_22324\1677906011.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unique_id'] = (df['player_id'] + '_' + df['season'].astype(str) + '_' + df['cleaned_name'] + '_' + df['std_pos']).str.replace(' ', '_')


## Export Datasets for more specific cleaning

In [ ]:
cleaned_dfs = [(rw_clean, 'rw_clean'), (rs_clean, 'rs_clean'), (ss_clean, 'ss_clean'), (inj_clean, 'inj_clean')]

for df, name in cleaned_dfs:
  df.to_csv(f'{name}_no_fb.csv', index=False)